## RAG 기본 파이프라인(1~8단계)


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI


아래는 기본적인 RAG 구조 이해를 위한 뼈대코드(skeleton code) 입니다.

각 단계별 모듈의 내용을 앞으로 상황에 맞게 변경하면서 문서에 적합한 구조를 찾아갈 수 있습니다.

(각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.)

In [2]:

#실습용 AOAI 환경변수 읽기
import os

AOAI_ENDPOINT=os.getenv("AOAI_ENDPOINT")
AOAI_API_KEY=os.getenv("AOAI_API_KEY")
AOAI_DEPLOY_GPT4O=os.getenv("AOAI_DEPLOY_GPT4O")
AOAI_DEPLOY_GPT4O_MINI=os.getenv("AOAI_DEPLOY_GPT4O_MINI")
AOAI_DEPLOY_EMBED_3_LARGE=os.getenv("AOAI_DEPLOY_EMBED_3_LARGE")
AOAI_DEPLOY_EMBED_3_SMALL=os.getenv("AOAI_DEPLOY_EMBED_3_SMALL")
AOAI_DEPLOY_EMBED_ADA=os.getenv("AOAI_DEPLOY_EMBED_ADA")

In [3]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("./data/extract_text/AI_Paradigm_Shift_Driven_by_DeepSeek.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 46


페이지의 내용을 출력합니다.

In [4]:
print(docs[10].page_content)

딥시크가 촉발한 AI 패러다임 변화와 플랫폼 정책방향
10
l (애플리케이션) ’24.2월 기존 바드(Bard)를 통합한 생성형 AI 앱 
‘제미나이’ 출시, 다양한 기능을 지속 탑재하여 성능 개선
- 생산성 플랫폼 Workspace, 웹브라우저 Chrome 등에 제미나이 통합
- 이용자 질문에 대한 심층연구 실행을 돕는 AI 에이전트 ‘제미나이 
딥 리서치’ 출시
- ’24.5월 차세대 범용 AI 에이전트 ‘프로젝트 아스트라’ 공개, ’25년 
상용화 예정
- 검색에서 사용자와 상호작용 및 후속질문이 가능한 AI기반 챗봇 
및 멀티모달 방식을 도입할 계획
- 검색에서 실시간 비디오 처리나 연구보고서 생성도 추진
◆ 메타
l (기초 모델) LLaMA를 연구 용도 중심의 비영리 라이센스로 전 세계 
AI 커뮤니티에 제공하고 개방형 협력을 구하는 전략으로 접근
l (애플리케이션) SNS 관련 앱 전반에 AI를 직접 통합하여 광범위한 
이용자 데이터를 바탕으로 새로운 형태의 광고 도구를 제공하는 
플랫폼으로 확장시킬 계획
- ’24.4월 멀티모달 AI 비서 '메타 AI'를 무료로 공개하여 페이스북, 
인스타그램, 왓츠앱 등을 통해 사용할 수 있음
- ’24.7월 AI 캐릭터와 챗봇을 만들 수 있는 ‘AI 스튜디오' 출시


`metadata` 를 확인합니다.

In [5]:
docs[10].__dict__

{'id': None,
 'metadata': {'producer': 'Hancom PDF 1.3.0.509',
  'creator': 'Hancom PDF 1.3.0.509',
  'creationdate': '2025-02-10T17:02:50+09:00',
  'source': '../data/extract_text/AI_Paradigm_Shift_Driven_by_DeepSeek.pdf',
  'file_path': '../data/extract_text/AI_Paradigm_Shift_Driven_by_DeepSeek.pdf',
  'total_pages': 46,
  'format': 'PDF 1.4',
  'title': '',
  'author': 'Insung-9',
  'subject': '',
  'keywords': '',
  'moddate': '2025-02-10T17:02:50+09:00',
  'trapped': '',
  'modDate': "D:20250210170250+09'00'",
  'creationDate': "D:20250210170250+09'00'",
  'page': 10},
 'page_content': "딥시크가 촉발한 AI 패러다임 변화와 플랫폼 정책방향\n10\nl (애플리케이션) ’24.2월 기존 바드(Bard)를 통합한 생성형 AI 앱 \n‘제미나이’ 출시, 다양한 기능을 지속 탑재하여 성능 개선\n- 생산성 플랫폼 Workspace, 웹브라우저 Chrome 등에 제미나이 통합\n- 이용자 질문에 대한 심층연구 실행을 돕는 AI 에이전트 ‘제미나이 \n딥 리서치’ 출시\n- ’24.5월 차세대 범용 AI 에이전트 ‘프로젝트 아스트라’ 공개, ’25년 \n상용화 예정\n- 검색에서 사용자와 상호작용 및 후속질문이 가능한 AI기반 챗봇 \n및 멀티모달 방식을 도입할 계획\n- 검색에서 실시간 비디오 처리나 연구보고서 생성도 추진\n◆ 메타\nl (기초 모델) LLaMA를 연구 용도 중심의 비영리 라이센스로

In [6]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 86


In [7]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = AzureOpenAIEmbeddings(
    model=AOAI_DEPLOY_EMBED_3_LARGE,
    openai_api_version="2024-02-01",
    api_key= AOAI_API_KEY,
    azure_endpoint=AOAI_ENDPOINT
    )

In [8]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [9]:
for doc in vectorstore.similarity_search("메타"):
    print(doc.page_content)

-4,967
-4,646
영업이익(비율)
23,365(48%)
16,384(41%)
 * 페이스북, 인스타그램, 왓츠앱 등으로 주요 수익원은 광고임
 ** 가상현실/증강현실 관련 하드웨어, 소프트웨어, 콘텐츠 사업
l (AI 투자) 메타의 CEO와 투자자들은 여전히 인프라에 많은 투자를 
하는 것이 전략적 이점이 될 것이라고 주장8)
※ 650억 달러(93조원) 투자해 130만개 규모 GPU를 데이터센터에 추가할 계획
- 전년 대비 50% 이상 늘리는 것이고, 시장 추정치를 30% 상회
- 모델의 비용 효율성 증가로 컴퓨팅 자원 수요가 줄어들기보다는 
더 나은 추론을 할 수 있도록 돕는 방향으로 이동할 것으로 전망
8) The Verge(2025.01.30), “Mark Zuckerberg says Meta isn’t worried about DeepSeek”.
딥시크가 촉발한 AI 패러다임 변화와 플랫폼 정책방향
32
◆ 메타
l (24.4분기 사업실적) 매출은 전년동기 대비 25% 증가한 401억 달러, 
순이익은 전년동기 대비 3배 이상 증가한 140억 달러 기록 
- 매출성장의 원동력은 전자상거래, 엔터테인먼트, 게임 분야이며 특히 
광고 수익은 최근 성장하는 테무, 쉬인 등 중국 광고주로부터의 수익이 
크게 증가한 것으로 보임
[표3] 메타2024년도4분기부문별매출
(단위: 백만달러)
구분
2024년 4분기
2023년 3분기
광고
46,783
38,706
기타
519
334
  Family of Apps* 합산
47,302
39,040
  Reality Labs**
1,083
1,071
총매출
48,385
40,111
  Family of Apps 영업이익
28,332
21,030
  Reality Labs 영업이익
-4,967
-4,646
영업이익(비율)
23,365(48%)
16,384(41%)
2024년 4분기
2023년
2024년
총매출
169,961
187,792
574,785
637,959
운영비용
156,752
166,589
537,93

In [10]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.

In [11]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("메타의 24분기 사업실적은?")

[Document(id='139abffe-e875-454f-afda-a512e3a33eb4', metadata={'producer': 'Hancom PDF 1.3.0.509', 'creator': 'Hancom PDF 1.3.0.509', 'creationdate': '2025-02-10T17:02:50+09:00', 'source': '../data/extract_text/AI_Paradigm_Shift_Driven_by_DeepSeek.pdf', 'file_path': '../data/extract_text/AI_Paradigm_Shift_Driven_by_DeepSeek.pdf', 'total_pages': 46, 'format': 'PDF 1.4', 'title': '', 'author': 'Insung-9', 'subject': '', 'keywords': '', 'moddate': '2025-02-10T17:02:50+09:00', 'trapped': '', 'modDate': "D:20250210170250+09'00'", 'creationDate': "D:20250210170250+09'00'", 'page': 32}, page_content='딥시크가 촉발한 AI 패러다임 변화와 플랫폼 정책방향\n32\n◆ 메타\nl (24.4분기 사업실적) 매출은 전년동기 대비 25% 증가한 401억 달러, \n순이익은 전년동기 대비 3배 이상 증가한 140억 달러 기록 \n- 매출성장의 원동력은 전자상거래, 엔터테인먼트, 게임 분야이며 특히 \n광고 수익은 최근 성장하는 테무, 쉬인 등 중국 광고주로부터의 수익이 \n크게 증가한 것으로 보임\n[표3]\x01 메타2024년도4분기부문별매출\n(단위: 백만달러)\n구분\n2024년 4분기\n2023년 3분기\n광고\n46,783\n38,706\n기타\n519\n334\n  Family of Apps* 합산\n47,302\n39,040\n  Reality Labs**\n1,083\n1,071\n총매출\n48,3

In [12]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [14]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.

# ChatOpenAI 언어 모델을 초기화합니다. temperature는 0으로 설정합니다.
llm = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment=AOAI_DEPLOY_GPT4O_MINI,
    temperature=0.0,
    api_key= AOAI_API_KEY,  
    azure_endpoint=AOAI_ENDPOINT
)


In [15]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

생성된 체인에 쿼리(질문)을 입력하고 실행합니다.

In [16]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "메타의 4분기 영업 실적은?"
response = chain.invoke(question)
print(response)

메타의 2024년도 4분기 영업 실적은 매출이 401억 달러로 전년 동기 대비 25% 증가하였고, 순이익은 140억 달러로 전년 동기 대비 3배 이상 증가하였습니다.


## 전체 코드

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("./data/extract_text/AI_Paradigm_Shift_Driven_by_DeepSeek.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = AzureOpenAIEmbeddings(
    model=AOAI_DEPLOY_EMBED_3_LARGE,
    openai_api_version="2024-02-01",
    api_key= AOAI_API_KEY,
    azure_endpoint=AOAI_ENDPOINT
    )

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.

# ChatOpenAI 언어 모델을 초기화합니다. temperature는 0으로 설정합니다.
# ChatOpenAI 언어 모델을 초기화합니다. temperature는 0으로 설정합니다.
llm = AzureChatOpenAI(
    openai_api_version="2024-02-01",
    azure_deployment=AOAI_DEPLOY_GPT4O_MINI,
    temperature=0.0,
    api_key= AOAI_API_KEY,  
    azure_endpoint=AOAI_ENDPOINT
)


# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "메타의 4분기 영업 실적은?"
response = chain.invoke(question)
print(response)